In [0]:
import sksurv as sks
import sksurv.preprocessing
import sksurv.metrics
import sksurv.datasets
import sksurv.linear_model

from pathlib import Path
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pymc_bart as pmb
import pandas as pd
import importlib
import numpy as np
import sklearn as skl
import scipy.stats as sp
plt.ioff()

In [0]:
importlib.reload(np.random)

In [0]:
# del np.random.seed
np.random.seed(99)

In [0]:
# #simple bernoulli time
# t = np.repeat([0,1,2],50)
# goof = sp.norm.rvs(0,1, size=100)

# # p_train = np.repeat([.9, .2], 50)

# # p true
# p = 0.9 - t*0.7

# # y
# y_train = sp.bernoulli.rvs(p)
# print(y_train.shape)
# print(t.shape)

# x_train = pd.DataFrame({"t":t,"goof":goof})
# x_train
# y_train


In [0]:
# # t_s = np.unique(x_train["t"]).shape[0]
# x_train["t2"] = np.where(x_train["t"] == 0, "a", "b")
# t_idx, t_code = pd.factorize(x_train["t2"])
# x_train["t3"] = t_idx
# # t_code
# # t_code
# t_s = len(t_code)
# t_idx

In [0]:
# with pm.Model() as bart:
#     # x_data = pm.MutableData("x", x_train)
#     # y_data = pm.MutableData("y", delta1)
    
#     # f = pmb.BART("f", X = x_data, Y = zdelt, m=50)
#     f = pmb.BART("f", X = x_train[["t3", "goof"]], Y = y_train, m=20, shape=t_s)
#     # z = pm.Deterministic("z", )

#     mu = pm.Deterministic("mu", pm.math.invprobit(f))
#     # y_pred = pm.Binomial("y_pred", p = mu, observed = delta2)
    
#     y_pred = pm.Bernoulli("y_pred", p=(mu[t_idx]), observed=y_train)
#     # y_pred = pm.Bernoulli("y_pred", p=(mu), observed=y_train, shape=x_data.shape[0])
#     smp4 = pm.sample(random_seed=2, draws=100, compute_convergence_checks=False)

In [0]:
# smp4

In [0]:
# coords = {"t":[0,1]}
# with pm.Model(coords = coords) as bart:
#     # x_data = pm.MutableData("x", x_train)
#     # y_data = pm.MutableData("y", delta1)
    
#     # f = pmb.BART("f", X = x_data, Y = zdelt, m=50)
#     f = pmb.BART("f", X = x_train[["t", "goof"]], Y = y_train, m=20, shape=t_s)
#     # z = pm.Deterministic("z", )

#     mu = pm.Deterministic("mu", pm.math.invprobit(f))
#     # y_pred = pm.Binomial("y_pred", p = mu, observed = delta2)
    
#     y_pred = pm.Bernoulli("y_pred", p=(mu[t_idx]), observed=y_train)
#     # y_pred = pm.Bernoulli("y_pred", p=(mu), observed=y_train, shape=x_data.shape[0])
#     smp4 = pm.sample(random_seed=2, draws=100, compute_convergence_checks=False)

In [0]:
# bmi = pd.read_csv(pm.get_data("bmi.csv"))
# bmi

In [0]:
# y = bmi.bmi.values
# X = bmi.age.values[:, None]


# y_stack = np.stack([bmi.bmi.values] * 3)
# quantiles = np.array([[0.1, 0.5, 0.9]]).T
# quantiles
# print(y.shape)
# print(X.shape)
# print(y_stack.shape)
# print(quantiles.shape)


In [0]:
# X = np.linspace(0, 1, 120)[:,None]
# Y = np.repeat([0, 1, 2], 40)
# plt.plot(X[:,0], Y, ".")

In [0]:
# X.shape
# Y

In [0]:
# with pm.Model() as bart_model:
#     μ = pmb.BART("μ", X=X, Y=Y, m=50, shape=(3, 120))
#     θ = pm.Deterministic('θ', pm.math.softmax(μ, axis=0))
#     y = pm.Categorical("y", p=θ.T, observed=Y)
#     idata = pm.sample()

In [0]:
# idata

In [0]:
# posterior_mean = az.extract(idata, var_names='θ').mean("sample")
# posterior_mean[:,0:10]
# # idata.posterior["θ"][0][0].values[:,60:70]

In [0]:
# draws = smp4.posterior["mu"].values.reshape(4000, 100)
# m_draws = np.mean(draws, axis=0)

# print(np.mean(m_draws[0:50]))
# print(np.mean(m_draws[50:100]))




In [0]:
# PH: 𝛼 = 2.0, 𝜆 = exp{3 + 0.1(x1 + x2 + x3 + x4 + x5 + x6) + x7}
#S(t|𝛼, 𝜆) = e ^−(t∕𝜆)𝛼
def sim_surv(N=100, 
            T=100, 
            x_vars = 1, 
            lambda_f=None, 
            a=2, 
            alpha_f = None, 
            seed=999, 
            cens_ind = True,
            cens_scale = 20,
            err_ind = False):
    # np.random.seed(seed)

    x_mat = np.zeros((N, x_vars))
    for x in np.arange(x_vars):
        x1 = sp.bernoulli.rvs(.5, size = N)
        x_mat[:,x] = x1
    # calculate lambda
    
    # set lambda
    if lambda_f is None:
        lmbda = np.exp(2 + 0.3*(x_mat[:,0] + x_mat[:,1]) + x_mat[:,2])
    else:
        lmbda = eval(lambda_f)
    
    # set alpha if specified
    if alpha_f is None:
        a = np.repeat(a, N)
    else:
        a = eval(alpha_f)

    # add error
    if err_ind:
        error = sp.norm.rvs(0, .5, size = N)
        lmbda=lmbda + error

    # get time series
    t = np.linspace(0,T, T)

    # calculate survival and event times
    sv_mat = np.zeros((N, t.shape[0]))
    tlat = np.zeros(N)
    for idx, l in enumerate(lmbda):
        sv = np.exp(-1 * np.power((t/l), a[idx]))
        sv_mat[idx,:] = sv
        
        # generate event times 
        unif = np.random.uniform(size=1)
        ev = lmbda[idx] * np.power((-1 * np.log(unif)), 1/a[idx])
        tlat[idx] = ev

    if cens_ind:
        # censor
        cens = np.ceil(np.random.exponential(size = N, scale = cens_scale))

        # min cen and surv event
        t_event  = np.minimum(cens, np.ceil(tlat))
        status = (tlat <= cens) * 1
    else:
        cens=np.zeros(N)
        t_event = np.ceil(tlat)
        status = np.ones(N)

        

    return sv_mat, x_mat, lmbda, a, tlat, cens, t_event, status

# np.power()

In [0]:
# sv_mat, x_mat = sim_ph(N=10, T=10, x_vars=3, err_ind = True)
# sv_mat, x_mat = sim_surv(N=10, 
#                     T=10,
#                     x_vars=4, 
#                     f = "np.exp(2 + 0.3*(x_mat[:,0] + x_mat[:,1]) + x_mat[:,2]) + x_mat[:,3]",
#                     err_ind = True)

T = 10
sv_mat, x_mat, lmbda, a, tlat, cens, t_event, status = sim_surv(N=100, 
                    T=T,
                    x_vars=2,
                    a = 2,
                    lambda_f = "np.exp(1 + 0.3*(x_mat[:,0] + x_mat[:,1]))",
                    cens_ind=False,
                    cens_scale=60,
                    err_ind = False)

# sv_mat, x_mat = sim_surv(N=10, 
#                     T=100,
#                     x_vars=1, 
#                     f = "20 + x_mat[:,0]",
#                     alpha_f="0.7 + 1.3*x_mat[:,0]",
#                     err_ind = True)



In [0]:
# print(lmbda)
# print(a)
# print(sv_mat)
# print(x_mat)
print(tlat)
print(cens)
print(t_event)
print(status)

In [0]:
# t_event

In [0]:
# x_out, x_idx, x_cnt = get_x_info(x_mat)
# print(x_out, x_idx, x_cnt)
# str(list(zip(x_out, x_cnt)))

In [0]:
def get_x_info(x_mat):
    x = np.unique(x_mat, axis=0, return_index=True, return_counts=True)
    x_out, x_idx, x_cnt = x[0], x[1], x[2]
    return x_out, x_idx, x_cnt

# x_info = get_x_info(x_mat)
# str(x_info)

def get_status_perc(status):
    out = status.sum()/status.shape[0]
    cens = 1-out
    return out, cens
# event_calc, cens_calc = get_status_perc(status)

def get_event_time_metric(t_event):
    t_mean = t_event.mean()
    t_max = t_event.max()
    return t_mean, t_max


# print(t_mean, t_max)

def get_train_matrix(x_mat, t_event, status):
    et = pd.DataFrame({"status": status, "time":t_event})
    train = pd.concat([et, pd.DataFrame(x_mat)],axis=1)
    return train

def get_y_sklearn(status, t_event):
    y = np.array(list(zip(np.array(status, dtype="bool"), t_event)), dtype=[("Status","?"),("Survival_in_days", "<f8")])
    return y



# et
# pd.DataFrame(x_mat)

In [0]:
x_out, x_idx, x_cnt = get_x_info(x_mat)
t_mean, t_max = get_event_time_metric(t_event)
train = get_train_matrix(x_mat, t_event, status)
y = get_y_sklearn(status, t_event)

# print(x_out)
# print(x_idx)
# print(x_cnt)
# y

In [0]:
# # check on how to set cens_scale to encounter a certain amount of censoring
# its=100
# cens_sum = np.zeros(its)
# lmbda_sum = np.zeros(its)
# for i in np.arange(its):
#     sv_mat, x_mat, lmbda, a, tlat, cens, t_event, status = sim_surv(N=100, 
#                     T=10,
#                     x_vars=2,
#                     a = 3,
#                     lambda_f = "np.exp(2 + 0.3*(x_mat[:,0] + x_mat[:,1]))",
#                     cens_scale=38,
#                     err_ind = False)
#     lmbda_sum[i] = lmbda.mean()
#     cens_sum[i] = status.sum()

# print(100-cens_sum.mean())
# print(lmbda_sum.mean())

In [0]:
# # eval("np.exp(2)")
# t = np.arange(10)
# for idx in np.arange(sv_mat.shape[0]):
#     plt.plot(t, sv_mat[idx,:], label = str(x_mat[idx,:]))
#     plt.legend()

# # str(x_mat[0,:])

In [0]:
# # Simulation study

# its=10

# # for i in np.arange(its):
# sv_mat, x_mat, lmbda, a, tlat, cens, t_event, status = sim_surv(
#                 N=2000, 
#                 T=20,
#                 x_vars=2,
#                 a = 3,
#                 lambda_f = "np.exp(2 + 0.3 *x_mat[:,0] + .7 * x_mat[:,1])",
#                 cens_scale=38,
#                 cens_ind=False,
#                 err_ind = False)



# # t = np.arange(10)
# # for idx in np.arange(sv_mat.shape[0]):
# #     plt.plot(t, sv_mat[idx,:], label = str(x_mat[idx,:]))
# #     plt.legend()





In [0]:

# def plot_sv(x_mat, sv_mat, t, title="TITLE", save=False, dir = ".", show=False):
#     dist_x, dist_idx = np.unique(x_mat, axis=0, return_index=True)

#     # print(tt)
#     try:
#         fig = plt.figure()
#         for idx, i in enumerate(sv_mat[dist_idx]):
#             plt.plot(np.arange(t), i, label = str(dist_x[idx]))
#             plt.legend()
#             plt.title(title)
#         if show:
#             plt.show()
#         if save:
#             plt.savefig(f"{dir}/{title}.png")
#     finally:
#         plt.close(fig)

# plot_sv(x_mat, cph_fx, 20, show=False)
# for idx, i in enumerate(cph_fx)
# cph_fx[1]
def plot_sv(x_mat, sv_mat, t, title="TITLE", save=False, dir=".", show=False):
    dist_x, dist_idx = np.unique(x_mat, axis=0, return_index=True)
    if type(t) == int:
        print("here")
        tt = np.arange(t)
    else:
        tt = t

    # print(tt)
    try:
        fig = plt.figure()
        if len(sv_mat) != len(dist_idx):
            for idx, i in enumerate(sv_mat[dist_idx]):
                plt.plot(tt, i, label = str(dist_x[idx]))
                plt.legend()
                plt.title(title)
        else:
            for idx, i in enumerate(sv_mat):
                # plt.step(i.x, i.y, label = str(dist_x[idx]))
                plt.plot(tt, i, label = str(dist_x[idx]))
                plt.legend()
                plt.title(title)
        if show:
            plt.show()
        if save:
            plt.savefig(f"{dir}/{title}.png")
    finally:
        plt.close(fig)

In [0]:
# plot_sv(x_mat, sv_mat, t=T, show=True)
# np.arange(T)
plot_sv(x_mat, sv_mat[x_idx], t=T, show=True)
# type(T)


In [0]:
# # get kpm
# kpm = sks.nonparametric.kaplan_meier_estimator(np.array(status, dtype="bool"), t_event)
# plt.plot(kpm[0], kpm[1])

In [0]:
# cox model
# y = np.array(list(zip(np.array(status, dtype="bool"), t_event)), dtype=[("Status","?"),("Survival_in_days", "<f8")])
cph = sks.linear_model.CoxPHSurvivalAnalysis()
cph.fit(train.iloc[:,2:], y)
cph_surv = cph.predict_survival_function(pd.DataFrame(x_out))

In [0]:
# get time and probs
cph_sv_t = cph_surv[0].x
cph_sv_val = [sf(cph_sv_t) for sf in cph_surv]
# add a time 0 with prob 1 
cph_sv_t = np.concatenate([np.array([0]), cph_sv_t])
cph_sv_val = [np.concatenate([np.array([1]), sv]) for sv in cph_sv_val]

plot_sv(x_mat, cph_sv_val, t=cph_sv_t, show=True)

In [0]:
# cph_surv[0].y

In [0]:
# cph_fx = [sf(np.arange(sf.x.max())) for sf in cph_surv]
# cph_fx[3]
# cph_surv

In [0]:
# plot_sv(x_mat, cph_fx, 20, show=False)
# for idx, i in enumerate(cph_fx)
# cph_fx[1]

# def plot_sv(x_mat, sv_mat, t, title="TITLE", save=False, dir=".", show=False):
#     dist_x, dist_idx = np.unique(x_mat, axis=0, return_index=True)

#     # print(tt)
#     try:
#         fig = plt.figure()
#         if len(sv_mat) != len(dist_idx):
#             for idx, i in enumerate(sv_mat[dist_idx]):
#                 plt.step(np.arange(t), i, label = str(dist_x[idx]))
#                 plt.legend()
#                 plt.title(title)
#         else:
#             for idx, i in enumerate(sv_mat):
#                 # plt.step(i.x, i.y, label = str(dist_x[idx]))
#                 plt.step(np.arange(t), i, label = str(dist_x[idx]))
#                 plt.legend()
#                 plt.title(title)
#         if show:
#             plt.show()
#         if save:
#             plt.savefig(f"{dir}/{title}.png")
#     finally:
#         plt.close(fig)

In [0]:
# plot_sv(x_mat, cph_surv, 20, show= True)
# cph_surv[0].x.max()
# # cph_fx[0].shape
# cph_t = cph_surv[0].x.max()
# plot_sv(x_mat, cph_fx, t=cph_t, show= True)


In [0]:
# time_points = np.linspace(0,20, 100)
# for i, surv_func in enumerate(cph_surv):
#     plt.step(time_points, surv_func(time_points), where="post", label=f"Sample {str(dist_x[i])}")
# plt.ylabel("est. Surv")
# plt.xlabel("time")
# plt.legend(loc="best")


In [0]:
# try with the rsf
import sksurv.ensemble

rsf = sks.ensemble.RandomSurvivalForest(
    n_estimators=1000, min_samples_split=10, min_samples_leaf=15, n_jobs=-1, random_state=20
)
rsf.fit(pd.DataFrame(x_mat), y)

In [0]:
rsf_surv1 = rsf.predict_survival_function(pd.DataFrame(x_out))

In [0]:
# get the predictions out
rsf_t = rsf_surv1[0].x
print(rsf_t)
rsf_fx = [sf(rsf_t) for sf in rsf_surv1]
print(rsf_fx)
# add a time 0 with prob 1 
rsf_t = np.concatenate([np.array([0]), rsf_t])
rsf_fx = [np.concatenate([np.array([1]), sv]) for sv in rsf_fx]

In [0]:
plot_sv(x_mat, rsf_fx, t = rsf_t, show=True)

In [0]:
# time_points = np.linspace(0,20, 100)
# for i, surv_func in enumerate(rsf_surv1):
#     plt.step(time_points, surv_func(time_points), where="post", label=f"Sample {dist_x[i]}")
# plt.ylabel("est. Surv")
# plt.xlabel("time")
# plt.legend(loc="best")

# BART

In [0]:

def surv_pre_train2(data_x_n, data_y, X_TIME=True):
    # set up times
    # t_sort = np.append([0], np.unique(data_y["Survival_in_days"]))
    t_sort = np.unique(data_y["Survival_in_days"])
    t_ind = np.arange(0,t_sort.shape[0])
    t_dict = dict(zip(t_sort, t_ind))

    # set up delta
    delta = np.array(data_y["Status"], dtype = "int")
    
    t_out = []
    pat_x_out = []
    delta_out = []
    for idx, t in enumerate(data_y["Survival_in_days"]):
        # get the pat_time and use to get the array of times for the patient
        p_t_ind = t_dict[t]
        p_t_set = t_sort[0:p_t_ind+1]
        t_out.append(p_t_set)
        
        size = p_t_set.shape[0]
        # get patient array
        pat_x = np.tile(data_x_n.iloc[idx].to_numpy(), (size, 1))
        pat_x_out.append(pat_x)

        # get delta
        pat_delta = delta[idx]
        delta_set = np.zeros(shape=size, dtype=int)
        delta_set[-1] = pat_delta
        delta_out.append(delta_set)
    
    
    t_out, delta_out, pat_x_out = np.concatenate(t_out), np.concatenate(delta_out), np.concatenate(pat_x_out)
    if X_TIME:
        pat_x_out = np.array([np.concatenate([np.array([t_out[idx]]), i]) for idx, i in enumerate(pat_x_out)])
    return t_out, delta_out, pat_x_out

# def surv_pre_test2(data_x_n, data_y, T, X_TIME=True):
#     # t_sort = np.append([0], np.unique(data_y["Survival_in_days"]))
#     t_sort = np.unique(data_y["Survival_in_days"])
#     t_out = []
#     pat_x_out = []
#     for idx, t in enumerate(data_y["Survival_in_days"]):
#         # get the pat_time and use to get the array of times for the patient
#         p_t_set = t_sort
#         t_out.append(p_t_set)
        
#         size = p_t_set.shape[0]
#         # get patient array
#         pat_x = np.tile(data_x_n.iloc[idx].to_numpy(), (size, 1))
#         pat_x_out.append(pat_x)
    
#     t_out, pat_x_out = np.concatenate(t_out),  np.concatenate(pat_x_out)
#     if X_TIME:
#         pat_x_out = np.array([np.concatenate([np.array([t_out[idx]]), i]) for idx, i in enumerate(pat_x_out)])
#     return t_out, pat_x_out



# def get_bart_test(x_out, T):
#     d1 = np.arange(T + 1)
#     d2 = np.arange(x_out.shape[1])
    
#     out = np.stack(np.array(np.meshgrid(d1, d2, d2)),-1).reshape(-1, d2.shape[0] + 1)
#     return out


In [0]:
def get_bart_test(x_out, T):
    s0 = x_out.shape[0]
    s1 = x_out.shape[1]
    # create time range
    # d1 = np.arange(T+1)
    d1 = T
    # repeating time range
    d2 = np.tile(d1,s0).reshape(d1.shape[0]*s0,1)
    # repeat x_out and shape as long by nvar
    d3 = np.tile(x_out, d1.shape[0]).reshape(s0*d1.shape[0], s1)
    # holding matrix
    d4 = np.matrix(np.zeros((d3.shape[0], d3.shape[1] + 1))) # always +1 because only adding on time col
    # replace
    d4[:,0] = d2
    d4[:,1:3] = d3
    return d4
# d_out = np.matrix()


In [0]:
train_t, train_delta, train_x = surv_pre_train2(pd.DataFrame(x_mat), y)
# test_t, test_x = surv_pre_test2(x, y)
off = sp.norm.ppf(np.mean(train_delta))

In [0]:
bart_t = np.unique(train_t).max()
# print(bart_t)
test_x = get_bart_test(x_out, np.unique(train_t))

In [0]:
with pm.Model() as bart:
    x_data = pm.MutableData("x", train_x)
    # y_data = pm.MutableData("y", delta1)
    
    f = pmb.BART("f", X = x_data, Y = train_delta, m=50)
    z = pm.Deterministic("z", f + off)

    mu = pm.Deterministic("mu", pm.math.invprobit(z))
    pm.Bernoulli("y_pred", p=(mu), observed=train_delta, shape=x_data.shape[0])
    bdata = pm.sample(random_seed=2, draws=1000, tune = 100, cores=8)

In [0]:
bdata

In [0]:
# test_x2 = np.unique(test_x, axis=0)
# # np.matrix(test_x2)[:,0]
# test_x.shape

In [0]:
with bart:
    pm.set_data({"x":pd.DataFrame(test_x)})
    pp = pm.sample_posterior_predictive(bdata, var_names = ["y_pred", "f", "z", "mu"])

In [0]:
pp

In [0]:
# get sv
def get_sv_fx(pp, x_out):
    p0 = pp.posterior_predictive["mu"].values.shape[0]
    p1 = pp.posterior_predictive["mu"].values.shape[1]
    p2 = pp.posterior_predictive["mu"].values.shape[2]
    r0 = x_out.shape[0]
    r1 = int(p2/r0)
    pp_post = pp.posterior_predictive["mu"].values.reshape(p0*p1, p2).mean(axis=0).reshape(r0, r1)
    sv = (1-pp_post).cumprod(axis=1)
    return sv

In [0]:
# shpe = pp.posterior_predictive["mu"].values.shape
# shpe0 = shpe[0] * shpe[1]
# shpe1 = shpe[2]
# shp_grps = x_out.shape[0]
# shpeT = int(shpe1/shp_grps)

# pp_post = pp.posterior_predictive["mu"].values.reshape((shpe0,shpe1))
# pp_postm = pp_post.mean(axis=0).reshape(shpeT, 4).T
# sv = (1-pp_postm).cumprod(axis=1)
# sv

sv_fx = get_sv_fx(pp, x_out)
sv_fx

In [0]:
# x_mat
# get the predictions out
sv_t = np.unique(train_t)
# print(rsf_t)
# print(sv)

# add a time 0 with prob 1 
sv_t = np.concatenate([np.array([0]), sv_t])
sv_fx2 = [np.concatenate([np.array([1]), sv]) for sv in sv_fx]
print(sv_t)
print(sv_fx2)

In [0]:
plot_sv(x_mat, sv_fx2, t = sv_t, show=True)

In [0]:
# pp_post = pp.posterior_predictive["mu"].values.reshape((800,64))
# pp_postm = pp_post.mean(axis=0).reshape(36,4).T
# sv = (1-pp_postm).cumprod(axis=1)

In [0]:
# for i in sv:
# test_str = test_x2[:,[1,2]][0:4]

# fig = plt.figure()
# for idx, i in enumerate(sv):
#     plt.step(np.arange(i.shape[0]), i, label = str(test_str[idx]))
#     plt.legend()
# plt.show()
# plt.close(fig)

# Metrics

In [0]:
def get_metrics(f_t, f, T, quants = np.array([0.1, 0.25, 0.5, 0.75, 0.9])):
    # t_quant = np.array(np.ceil(np.quantile(np.arange(T), quants)), dtype="int")
    t_quant = np.array(np.ceil(np.quantile(T, quants)), dtype="int")
    # print(t_quant)
    
    # f_t = np.matrix(rsf_fx)[:,t_quant]
    f_t = np.matrix(f_t)[:,t_quant]
    f = np.matrix(f)[:, t_quant]
    # f = np.matrix(sv_mat[dist_idx])[:, t_quant]

    rmse = np.round(np.sqrt(np.mean(np.power(f_t - f, 2), axis=0)), 4)
    bias = np.round(np.mean(f_t - f, axis = 0), 4)
    
    return rmse, bias, t_quant

In [0]:
# cph_fx
cph_sv_val

In [0]:
rsf_rmse, rsf_bias, t_quant = get_metrics( f_t = rsf_fx, f = sv_mat[x_idx], T = rsf_t)

cph_rmse, cph_bias, t_quant = get_metrics( f_t = cph_sv_val, f = sv_mat[x_idx], T = cph_sv_t)

bart_rmse, bart_bias, t_quant = get_metrics(f_t = sv_fx2, f = sv_mat[x_idx], T = sv_t)
print(rsf_rmse)
print(rsf_bias)
print(cph_rmse)
print(cph_bias)
print(bart_rmse)
print(bart_bias)

In [0]:
# print(sv_mat[x_idx])
# print(sv_fx2)

In [0]:
fig = plt.figure()
plt.plot(t_quant, rsf_rmse[0], label="rsf")
plt.plot(t_quant, cph_rmse[0], label = "cph")
plt.plot(t_quant, bart_rmse[0], label = "bart")
plt.legend()
plt.show()
plt.close(fig)

In [0]:
fig = plt.figure()
plt.plot(t_quant, rsf_bias[0], label="rsf")
plt.plot(t_quant, cph_bias[0], label = "cph")
plt.plot(t_quant, bart_bias[0], label = "bart")
plt.legend()
plt.show()
plt.close(fig)